# Smart Queue Monitoring System - Retail Scenario

## Overview
In this notebook, we request an **IEI Tank-870** edge node and run inference on the different hardware types (CPU, GPU, VPU, FPGA).

After the inference is completed, the output video and stats files are retrieved and stored.

## Step 0: Set Up

#### IMPORTANT: Set up paths so we can run Dev Cloud utilities
You *must* run this every time you enter a Workspace session.
(Tip: select the cell and use **Shift+Enter** to run the cell.)

In [ ]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

## Step 1 : Inference on a Video

In the next few cells, we submit a job using the `qsub` command and retrieve the results for each job. Each of the cells below should submit a job to different edge compute nodes.

The output of the cell is the `JobID` of your job, usefull to track progress of a job with `liveQStat`.

There is a submission job for each of the following hardware types:
* **CPU**
* **GPU**
* **VPU**
* **FPGA**


After submission, they will go into a queue and run as soon as the requested compute resources become available. 

If the job successfully runs and completes, it outputs a video and a stats text file in the `results/manufacturing/<DEVICE>` directory.

## Step 1.1: Submit to an Edge Compute Node with an Intel CPU
The script below submits a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload should run on the CPU.

In [ ]:
# Function to submit job
model_path = '/data/models/intel/person-detection-retail-0013/FP16/person-detection-retail-0013'
video_path = '/data/resources/retail.mp4'
queue_param = '/data/queue_param/retail.npy'
max_people = 2

def submit_job(device,node_name):
    device_name = device
    if device_name.startswith('HETERO:FPGA'):
        device_name = 'FPGA'
    elif device_name.startswith('MYRIAD'):
        device_name = 'VPU'
        
    
    output_path = '/output/results/retail/' + str.lower(device_name)
    flag = '{} {} {} {} {} {}'.format(model_path, device, video_path, queue_param, output_path, max_people)
    
    job_name = "retail_" + str.lower(device_name)
    
    job_ID = !qsub queue_job.sh -l nodes=1:{node_name} -d . -F "{flag}" -N job_name

    return job_ID

In [ ]:
#Submit job to the queue
cpu_job_id = submit_job('CPU', 'tank-870:i5-6500te')

print(cpu_job_id[0])

#### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job.

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

#### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results
get_results.getResults(cpu_job_id[0], filename='output.tgz', blocking=True)

#### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

#### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

#### View Output Video
Run the cell below to view the output video. If inference was successfully run, you should see a video with bounding boxes drawn around each person detected.

In [ ]:
import videoHtml

videoHtml.videoHTML('Retail CPU', ['results/retail/cpu/output_video.mp4'])

## Step 1.2: Submit to an Edge Compute Node with a CPU and IGPU
The script below submits a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500TE</a>. The inference workload should run on the **Intel® HD Graphics 530** integrated GPU.

In [ ]:
#Submit job to the queue
gpu_job_id = submit_job('GPU', 'tank-870:i5-6500te:intel-hd-530')

print(gpu_job_id[0])

### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job.

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

#### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results
get_results.getResults(gpu_job_id[0], filename='output.tgz', blocking=True)

#### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

#### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

#### View Output Video
Run the cell below to view the output video. If inference was successfully run, you should see a video with bounding boxes drawn around each person detected.

In [ ]:
import videoHtml

videoHtml.videoHTML('Retail GPU', ['results/retail/gpu/output_video.mp4'])

## Step 1.3: Submit to an Edge Compute Node with an Intel® Neural Compute Stick 2
The script below submits a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload should run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
#Submit job to the queue
vpu_job_id = submit_job('MYRIAD', 'tank-870:i5-6500te:intel-ncs2')

print(vpu_job_id[0])

### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job.

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

#### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results
get_results.getResults(vpu_job_id[0], filename='output.tgz', blocking=True)

#### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

#### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

#### View Output Video
Run the cell below to view the output video. If inference was successfully run, you should see a video with bounding boxes drawn around each person detected.

In [ ]:
import videoHtml

videoHtml.videoHTML('Retail VPU', ['results/retail/vpu/output_video.mp4'])

## Step 1.4: Submit to an Edge Compute Node with IEI Mustang-F100-A10
The script below submits a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core™ i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> FPGA card installed in this node.

In [ ]:
#Submit job to the queue
fpga_job_id = submit_job('HETERO:FPGA,CPU', 'tank-870:i5-6500te:iei-mustang-f100-a10')

print(fpga_job_id[0])

### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job.

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

#### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results
get_results.getResults(fpga_job_id[0], filename='output.tgz', blocking=True)

#### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

#### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

#### View Output Video
Run the cell below to view the output video. If inference was successfully run, you should see a video with bounding boxes drawn around each person detected.

In [ ]:
import videoHtml

videoHtml.videoHTML('Retail FPGA', ['results/retail/fpga/output_video.mp4'])

***Wait!***

Please wait for all the inference jobs and video rendering to complete before proceeding to the next step.

## Step 2: Assess Performance

Run the cells below to compare the performance across all 4 devices. The following timings for the model are being comapred across all 4 devices:

- Model Loading Time
- Average Inference Time
- FPS

In [ ]:
import matplotlib.pyplot as plt

device_list=['cpu', 'gpu', 'fpga', 'vpu']
inference_time=[]
fps=[]
model_load_time=[]

for device in device_list:
    with open('results/retail/'+device+'/stats.txt', 'r') as f:
        inference_time.append(float(f.readline().split("\n")[0].split(" ")[4]))
        fps.append(float(f.readline().split("\n")[0].split(" ")[43]))
        model_load_time.append(float(f.readline().split("\n")[0].split(" ")[4]))

In [ ]:
plt.bar(device_list, inference_time)
plt.xlabel("Device Used")
plt.ylabel("Total Inference Time in Seconds")
plt.show()

In [ ]:
plt.bar(device_list, model_load_time)
plt.xlabel("Device Used")
plt.ylabel("Model Loading Time in Seconds")
plt.show()